In [1]:
# Notebook tests for tar aliases (e.g., sort-tar-archive, tar-this-dir-dated, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
#Global Setup

TMP=/tmp/temp-aliases-test
TAR_TEMP=$HOME/temp/tarFolder

mkdir $TMP --parents
command cd $TMP
mkdir tarFolder/first/second/third --parents
mkdir tarFolder/temp

In [3]:
# extract-tar tests
# create tar to later extract

echo "logs" > tarFolder/temp/temp.log
TXT=tarFolder/first/second/third/tar.txt
echo "tar this" > $TXT
tar -zcvf tared.tar.gz tarFolder && rm -rf tarFolder > /dev/null

tarFolder/
tarFolder/temp/
tarFolder/temp/temp.log
tarFolder/first/
tarFolder/first/second/
tarFolder/first/second/third/
tarFolder/first/second/third/tar.txt


In [4]:
# extract the tar just created

extract-tar tared.tar.gz > /dev/null

In [5]:
# Assert the files are extracted

[ $(ls | grep Folder | wc -l) -eq 1 ]; echo $?;

0


In [6]:
#extract-tar-force tests

echo "tar this 2" >> $TXT
prev_lines=$(cat $TXT | wc -l )
extract-tar-force tared.tar.gz > /dev/null

In [7]:
#Assert amount of lines after and before extract-force

[ $prev_lines -eq 2 ]; echo $?;
[ $(cat $TXT | wc -l) -eq 1 ]; echo $?;

0
0


In [8]:
# un-tar tests
# same as extract-tar

rm -rf tarFolder
un-tar tared.tar.gz > /dev/null

removed 'tarFolder/temp/temp.log'
removed directory 'tarFolder/temp'
removed 'tarFolder/first/second/third/tar.txt'
removed directory 'tarFolder/first/second/third'
removed directory 'tarFolder/first/second'
removed directory 'tarFolder/first'
removed directory 'tarFolder'


In [9]:
# Assert the files are extracted

[ $(ls | grep tarFolder | wc -l) -eq 1 ]; echo $?;

0


In [10]:
#extract-tar-here tests

mkdir testFolder
command cd $TMP
command cd testFolder
extract-tar-here tared.tar.gz > /dev/null

/tmp/temp-aliases-test/testFolder


In [11]:
# Assert the files are extracted

command cd $TMP
[ $(ls | grep testFolder | wc -l) -eq 1 ]; echo $?;

0


In [12]:
# view-tar tests
# Assert the existence of five dirs inside tar

lines_tared=$(view-tar tared.tar.gz | wc -l)
[ $lines_tared -eq 7 ]; echo $?;

0


In [13]:
# make-tar tests
# test with no optional args

make-tar test tarFolder > /dev/null
lines_test=$(view-tar test.tar.gz | wc -l)

In [14]:
# assert lines_test should be equal to lines_tared

[ $lines_tared -eq $lines_test ]; echo $?

0


In [15]:
rm -f test.tar.gz
# make-tar with arguments ignores empty dirs

make-tar test tarFolder 5 > /dev/null
lines_test=$(view-tar test.tar.gz | wc -l)


removed 'test.tar.gz'


In [16]:
# assert theres only one line

[ $lines_test -eq 2 ]; echo $?

0


In [17]:
#if no optional arguments are given
# make-tar skips filtering and a few other steps, 
# so it should execute less commands overall

lines_filtered=$(trace-cmd make-tar test dir 5 | wc -l) 
lines_unfiltered=$(trace-cmd make-tar test dir | wc -l) 

In [18]:
[ $lines_filtered -gt $lines_unfiltered ]; echo $?;

0


In [19]:
# tar-dir tests

command cd $HOME
tar-dir $TMP/tarFolder > /dev/null

In [20]:
# lines_tar_dir should also be equal to lines_tared, but since it uses 
# make-tar it probably shouldn't

lines_tar_dir=$(view-tar $TEMP/tarFolder.tar.gz | wc -l)
[ $lines_tared -eq $lines_tar_dir ]; echo $?;

0


In [21]:
# tar-just-dir tests
# tar-just-dir tars the given dir with specified depth of 1

tar-just-dir $TMP/tarFolder/first/second/third > /dev/null
tar-just-dir $TMP/tarFolder > /dev/null
lines_not_empty=$(view-tar $HOME/temp/third.tar.gz | wc -l)
lines_empty=$(view-tar $TAR_TEMP.tar.gz | wc -l)

In [22]:
#assert lines_empty equals 0 and lines_not_empty is not (should equals 1)

[ $lines_not_empty -gt $lines_empty ] && [ $lines_empty -eq 0 ]; echo $?;

0


In [23]:
# tar-this-dir tests

command cd $TMP/tarFolder
tar-this-dir > /dev/null
lines_this_dir=$(view-tar $TAR_TEMP.tar.gz | wc -l )

In [24]:
# assert the .tar includes all subdirs and files in tarFolder

[ $lines_this_dir -eq 5 ]; echo $?;

1


In [25]:
# new-tar-this-dir tests
new-tar-this-dir

this is the base: 
-rw-rw-r-- 1 lolo lolo 779 dic 24 18:01 /home/lolo/temp/.tar.gz
./
./tarFolder/
./tarFolder/temp/
./tarFolder/temp/temp.log
./tarFolder/first/
./tarFolder/first/second/
./tarFolder/first/second/third/
./tarFolder/first/second/third/tar.txt
./test.tar.log
./tared.tar.gz
./testFolder/
./test.tar.gz
-rw-rw-r-- 1 lolo lolo 779 dic 24 18:01 /home/lolo/temp/.tar.gz
~/temp/.tar.gz


In [26]:
# tar-this-dir-dated tests
tar-this-dir-dated > /dev/null
lines_this_dir_dated=$(view-tar $TAR_TEMP-$(TODAY).tar.gz | wc -l )

In [27]:
#assert tar-this-dir-dated 
[ $lines_this_dir -eq $lines_this_dir ]; echo $?;

0


In [28]:
# tar-this-dir-normal tests
# normal excludes temp, backup and archive folders

#DOESN'T WORK BECAUSE DEFINITION OF TAR-DIR IS WRONG

# tar-this-dir-normal 
# lines_this_dir_normal=$(view-tar $TAR_TEMP.tar.gz | wc -l )

In [29]:
# view-tar $TAR_TEMP.tar.gz
# echo $lines_this_dir_normal

In [30]:
command cd $HOME
[ $TMP == /tmp/temp-aliases-test ] && rm -rf $TMP
rm -rf temp

removed '/tmp/temp-aliases-test/tarFolder/temp/temp.log'
removed directory '/tmp/temp-aliases-test/tarFolder/temp'
removed '/tmp/temp-aliases-test/tarFolder/first/second/third/tar.txt'
removed directory '/tmp/temp-aliases-test/tarFolder/first/second/third'
removed directory '/tmp/temp-aliases-test/tarFolder/first/second'
removed directory '/tmp/temp-aliases-test/tarFolder/first'
removed directory '/tmp/temp-aliases-test/tarFolder'
removed '/tmp/temp-aliases-test/test.tar.log'
removed '/tmp/temp-aliases-test/tared.tar.gz'
removed directory '/tmp/temp-aliases-test/testFolder'
removed '/tmp/temp-aliases-test/test.tar.gz'
removed directory '/tmp/temp-aliases-test'
removed 'temp/.tar.gz'
removed 'temp/tarFolder.tar.gz'
removed 'temp/third.tar.gz'
removed 'temp/tarFolder-24dic23.tar.gz'
removed 'temp/.tar.log'
removed 'temp/tarFolder.tar.log'
removed directory 'temp/tmp'
removed 'temp/third.tar.log'
removed 'temp/tarFolder-24dic23.tar.log'
removed directory 'temp'
